In [1]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device="cuda",
# )


# terminators = [
#     pipeline.tokenizer.eos_token_id,
#     pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]
model = AutoModelForCausalLM.from_pretrained(
model_id,
attn_implementation= "flash_attention_2",
# quantization_config=None,
# trust_remote_code=True,
torch_dtype = torch.bfloat16,
device_map = 'auto'
    )
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id,padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.bos_token_id



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# text = '''You are a cryptic crosswords expert. I will give you a clue. As you know, every clue has two parts: a definition and wordplay. Please extract the definition word/s from this clue. Notice that the definition usually comes at the clue's beginning or end. Just output the answer. 
# Clue: {clue}
# Output: 
# '''

text = """You are a cryptic crosswords expert. I will give you a clue and the answer. Please, explain how the answer was obtained.

Clue: {clue}

Answer: {answer}
"""

import pandas as pd
data = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/decrypting-crosswords/data/short_list_clues.csv')
responses = []
for i, row in data.iterrows():
        clue = row['Clue']
        answer = row['Answer']
    

        cur_clue = text.format(clue=clue, answer=answer)
        messages = [
        {"role": "user", "content": cur_clue},
        ]

        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )

        inputs = tokenizer(prompt,return_tensors="pt").to(model.device)
        inputs_length = inputs.input_ids.shape[1]


        outputs = model.generate(**inputs,
                                do_sample=False,
                                temperature=1.9,
                                top_p=0.9,
                                pad_token_id = tokenizer.eos_token_id)
        
        response  = tokenizer.batch_decode(outputs[:, inputs_length:], skip_special_tokens=True)[0]
        responses.append(response)
        print(response)
        # print(tokenizer.batch_decode(outputs[:, inputs_length:], skip_special_tokens=True))

with open('explanaitons_dasha_prompt.txt', 'w') as f:
    for response in responses:
        f.write(response + '\n')

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:500: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:505: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


A delicious-sounding clue!

Let's break it down:

* "Smear" is a verb that means to spread or apply a substance thinly.
* "pan" is a cooking vessel.
* "to cook cheese" suggests that the answer is related to cooking cheese.

Putting it all together, I think the clue is hinting at a type of pan that is used to cook cheese, and the answer is a type of cheese that is often cooked in a pan.

The answer, PARMESAN, is a type of Italian cheese that is often grated and cooked in a pan to melt and caramelize. The word "parmesan" can also be broken down into "par" (meaning "smear" or "spread") and "mesan" (a suffix suggesting a type of cheese). This etymological connection adds an extra layer of cleverness to the clue!

So, the answer PARMESAN is a perfect fit for the clue, and I'm sure the constructor of the crossword was delighted to have come up with such a clever and tasty clue!
A delightful cryptic crossword clue!

Let's break it down:

* "Agreed" is a hint that the answer is a phrase that m

In [24]:
print(prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You are a cryptic crosswords expert. I will give you a clue. As you know, every clue has two parts: a definition and wordplay. Please extract the wordplay type from this clue.
Here is a list of all possible wordplay types, and their descriptions:
- assemblage: the answer is broken into its component parts and the hint makes references to these in a sequence.
- anagram:  certain words or letters must be jumbled to form an entirely new term.
- hidden word: the answer will be hidden within one or multiple words within the provided phrase.
- container: the answer is broken down into different parts, with one part embedded within another.
- double definition: a word with two definitions.
only output the wordplay type.
Clue: Tricky problem, brewing artisan beer (5-6)
Output:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [1]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [19]:
from transformers import TextStreamer


text_streamer = TextStreamer(tokenizer)

text = '''You are a cryptic crosswords expert. I will give you a clue. As you know, every clue has two parts: a definition and wordplay. Please extract the definition word/s from this clue. Only output the definition word. 
clue: {clue}
definition: 
'''

clue = 'Smear pan to cook cheese (8)'

text = text.format(clue=clue)

inputs = tokenizer(text,
  return_tensors="pt")
inputs_length = inputs.input_ids.shape[1]
output = model.generate(**inputs, max_new_tokens = 32, do_sample = False)

# output = tokenizer.batch_decode(output[:, inputs_length:], skip_special_tokens=True)
output = tokenizer.batch_decode(output, skip_special_tokens=True)

print(output[0])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a cryptic crosswords expert. I will give you a clue. As you know, every clue has two parts: a definition and wordplay. Please extract the definition word/s from this clue. Only output the definition word. 
clue: Smear pan to cook cheese (8)
definition: 
Please extract the definition word/s from this clue. Only output the definition word. 

Answer: 
Pan 
Please let me know if this is correct. I


TypeError: list indices must be integers or slices, not tuple

In [41]:
text = '''You are a cryptic crossword expert. You are given a clue for a cryptic crossword. Only output the answer.
clue:
Promise to fix toilet pan (9)
output:
'''
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    tokenizer.convert_tokens_to_ids('<|end_of_text|>')
    
]
inputs = tokenizer(['sdfdsf','sdfdsf'],
  return_tensors="pt")
output = model.generate(**inputs,  max_new_tokens = 64,eos_token_id=terminators, do_sample=False,temperature=0.1, top_p= 0.9 )
print(tokenizer.decode(output[0], skip_special_tokens=False))



/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:500: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:505: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfds


In [42]:
output

tensor([[128000,     82,   3013,   5469,     69,    198,     82,   3013,   5469,
             69,    198,     82,   3013,   5469,     69,    198,     82,   3013,
           5469,     69,    198,     82,   3013,   5469,     69,    198,     82,
           3013,   5469,     69,    198,     82,   3013,   5469,     69,    198,
             82,   3013,   5469,     69,    198,     82,   3013,   5469,     69,
            198,     82,   3013,   5469,     69,    198,     82,   3013,   5469,
             69,    198,     82,   3013,   5469,     69,    198,     82,   3013,
           5469,     69,    198,     82,   3013,   5469],
        [128000,     82,   3013,   5469,     69,    198,     82,   3013,   5469,
             69,    198,     82,   3013,   5469,     69,    198,     82,   3013,
           5469,     69,    198,     82,   3013,   5469,     69,    198,     82,
           3013,   5469,     69,    198,     82,   3013,   5469,     69,    198,
             82,   3013,   5469,     69,    198,   

In [32]:
import re
def get_ans_words_chard(clue):
    # Regular expression to match strings inside parentheses
    pattern = r'\((.*?)\)'
    # Find all matches
    matches = re.findall(pattern, clue)[-1]

    numbers = matches.split(',')

    return len(numbers), matches



def crop_predictions(clues, predictions):

    lengthes = []
    for clue in clues:
        lengthes.append(get_ans_words_chard(clue)[1])

    cropped_predictions = []
    for i, pred in enumerate(predictions):
        cleaned_text = []
        pred_words = pred.split(' ')

        for word, length in zip(pred_words, lengthes[i]):
            cleaned_text.append(word[:length])
        cropped_predictions.append(' '.join(cleaned_text))

    return cropped_predictions




In [45]:
from utils import generate_prompt

example = { 'input': 'sdfsdf', 'target': 'sdfsdfsdf'}
print (generate_prompt(example, 'LLAMA3_BASE_PROMPT', True))



{'input': 'sdfsdf', 'target': 'sdfsdfsdf', 'prompt': 'You are a cryptic crossword expert. You are given a clue for a cryptic crossword. Output only the answer. \nclue:\nsdfsdf\noutput:\nsdfsdfsdf\n'}
